<center><h1>Assignment 05<br>
<b>Sentiment Analysis</b><br></center></h1>
<b>Problem:</b> Natural language processing has extensively applied in sentiment analysis as a
significant challenge. In this scenario, the objective is to determine if the tweets
shared by customers regarding technology companies that produce and sell
mobiles, computers, laptops, and similar products express positive sentiment or
negative sentiment.<br>
The goal will be to build a system that can accurately classify the new tweets
sentiments. You can divide the data into train and test. The Evaluation metric you
should use is the accuracy.

In [125]:
import pandas as pd
import numpy as np

In [126]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [127]:
data = pd.read_csv('/content/drive/MyDrive/datasets/tweets.csv')

In [128]:
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


## Text Preprocessing with NLTK

In [129]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [130]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [131]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [132]:
import re
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters and numbers, convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

In [133]:
data['tweet'] = data['tweet'].apply(preprocess_text)

In [134]:
data.head()

,id,label,tweet
0,1,0,fingerprint pregnancy test android apps beauti...
1,2,0,finally transparant silicon case thanks uncle ...
2,3,0,love would go talk makememories unplug relax i...
3,4,0,im wired know im george made way iphone cute d...
4,5,1,amazing service apple wont even talk question ...


## Train_Test Split

In [135]:
from sklearn.model_selection import train_test_split

In [136]:
x = data['tweet']
y = data['label']

In [137]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


## Bag of Words representation

In [138]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x_train_bow = cv.fit_transform(x_train)
x_test_bow = cv.transform(x_test)

In [139]:
from gensim.models import Word2Vec
tokenized_tweets = [tweet.split() for tweet in x_train]
word2vec_model = Word2Vec(sentences=tokenized_tweets, vector_size=100, window=5, min_count=1, workers=4)

In [140]:
# Function to get averaged Word2Vec vectors for tweets
def get_average_vector(tweet, model, vector_size):
    vector_sum = sum(model.wv[word] for word in tweet if word in model.wv)
    if vector_sum.any():
        return vector_sum / len(vector_sum)
    else:
        return [0] * vector_size

In [141]:
# Convert tweets to averaged Word2Vec vectors
x_train_vec = [get_average_vector(tweet.split(), word2vec_model, 100) for tweet in x_train]
x_test_vec = [get_average_vector(tweet.split(), word2vec_model, 100) for tweet in x_test]

# Convert to NumPy arrays for compatibility with SVM
x_train_vec = pd.DataFrame(x_train_vec).to_numpy()
x_test_vec = pd.DataFrame(x_test_vec).to_numpy()

## SVM

### SVM Model with Bag of Words

In [142]:
from sklearn.svm import SVC
from sklearn import metrics

In [143]:
svm_bow_mdl = SVC()
svm_bow_mdl.fit(x_train_bow, y_train)
y_pred_svm_bow = svm_bow_mdl.predict(x_test_bow)
accuracy_svm_bow = metrics.accuracy_score(y_test, y_pred_svm_bow)
print(f'SVM with Bag of Words Accuracy: {accuracy_svm_bow}')


SVM with Bag of Words Accuracy: 0.8926767676767676


### SVM Model with Word2Vec

In [144]:
x_train_vec

array([[-0.02905815,  0.04461781,  0.01061659, ..., -0.05000914,
         0.01256503, -0.01572662],
       [-0.01964477,  0.02655638,  0.00691631, ..., -0.0310523 ,
         0.00983629, -0.01440929],
       [-0.01566448,  0.02236398,  0.00484716, ..., -0.02599695,
         0.00651539, -0.00723531],
       ...,
       [-0.0248391 ,  0.03987034,  0.0086532 , ..., -0.04450922,
         0.01146205, -0.01284891],
       [-0.02065405,  0.03635317,  0.00907444, ..., -0.03942744,
         0.00714164, -0.00749701],
       [-0.03343914,  0.04993556,  0.01142206, ..., -0.05651421,
         0.01462796, -0.01642978]], dtype=float32)

In [145]:
svm_w2v_mdl = SVC()
svm_w2v_mdl.fit(x_train_vec, y_train)
y_pred_svm_w2v = svm_w2v_mdl.predict(x_test_vec)
accuracy_svm_w2v = metrics.accuracy_score(y_test, y_pred_svm_w2v)
print(f'SVM with Word2Vec Accuracy: {accuracy_svm_w2v}')

SVM with Word2Vec Accuracy: 0.7272727272727273


## Random Forest

In [146]:
from sklearn.ensemble import RandomForestClassifier

### Random Forest with BOW

In [147]:
rf_bow_mdl = RandomForestClassifier(n_estimators=100, random_state=42)
rf_bow_mdl.fit(x_train_bow, y_train)
y_pred_rf_bow = rf_bow_mdl.predict(x_test_bow)
accuracy_rf_bow = metrics.accuracy_score(y_test, y_pred_rf_bow)
print(f'Random Forest with Bag of Words Accuracy: {accuracy_rf_bow}')

Random Forest with Bag of Words Accuracy: 0.8724747474747475


### Random Forest with Worrd2Vec

In [148]:
rf_w2v_mdl =  RandomForestClassifier(n_estimators=100, random_state=42)
rf_w2v_mdl.fit(x_train_vec, y_train)
y_pred_rf_w2v = rf_w2v_mdl.predict(x_test_vec)
accuracy_rf_w2v = metrics.accuracy_score(y_test, y_pred_rf_w2v)
print(f'Random Forest with Word2Vec Accuracy: {accuracy_rf_w2v}')

Random Forest with Word2Vec Accuracy: 0.8175505050505051


## Neural Network with Word2Vec Embeddings

In [149]:
from keras.models import Sequential
from keras.layers import Dense,LSTM, Embedding, SimpleRNN,Dropout
from tensorflow.keras.preprocessing.text import Tokenizer

In [150]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

In [151]:
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

In [152]:
# to avoid the problem with different sizes

from keras.utils import pad_sequences
x_train_padded = pad_sequences(x_train_seq, maxlen=100)
x_test_padded = pad_sequences(x_test_seq, maxlen=100)

### LSTM

In [153]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim=128, input_length=100))
lstm_model.add(LSTM(10))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(50, activation='relu'))
lstm_model.add(Dense(1,activation='sigmoid'))

In [154]:
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [155]:
from keras.callbacks import EarlyStopping
lstm_model.fit(x_train_padded, y_train, epochs=10, validation_split=0.1,callbacks = EarlyStopping(patience=5))

Epoch 1/10
179/179 [==============================] - 18s 84ms/step - loss: 0.4286 - accuracy: 0.7922 - val_loss: 0.2614 - val_accuracy: 0.8833
Epoch 2/10
179/179 [==============================] - 14s 76ms/step - loss: 0.1954 - accuracy: 0.9263 - val_loss: 0.2774 - val_accuracy: 0.8927
Epoch 3/10
179/179 [==============================] - 14s 76ms/step - loss: 0.1084 - accuracy: 0.9681 - val_loss: 0.3323 - val_accuracy: 0.8738
Epoch 4/10
179/179 [==============================] - 15s 82ms/step - loss: 0.0706 - accuracy: 0.9793 - val_loss: 0.4154 - val_accuracy: 0.8596
Epoch 5/10
179/179 [==============================] - 16s 90ms/step - loss: 0.0466 - accuracy: 0.9863 - val_loss: 0.4874 - val_accuracy: 0.8549
Epoch 6/10
179/179 [==============================] - 16s 91ms/step - loss: 0.0313 - accuracy: 0.9919 - val_loss: 0.6091 - val_accuracy: 0.8470


In [156]:
y_pred_lstm = lstm_model.predict(x_test_padded)

50/50 [==============================] - 1s 9ms/step


In [157]:
y_pred_lstm[y_pred_lstm>0.5]=1
y_pred_lstm[y_pred_lstm<0.5]=0

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred_lstm, y_test)

0.8592171717171717

### RNN

In [158]:
rnn_model = Sequential()
rnn_model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim=128, input_length=100))
rnn_model.add(SimpleRNN(10))
rnn_model.add(Dropout(0.5))
rnn_model.add(Dense(50, activation='relu'))
rnn_model.add(Dense(1,activation='sigmoid'))

In [159]:
rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [160]:
rnn_model.fit(x_train_padded, y_train, epochs=10, validation_split=0.1,callbacks = EarlyStopping(patience=5))

Epoch 1/10
179/179 [==============================] - 13s 64ms/step - loss: 0.4963 - accuracy: 0.7583 - val_loss: 0.2852 - val_accuracy: 0.8912
Epoch 2/10
179/179 [==============================] - 8s 47ms/step - loss: 0.2305 - accuracy: 0.9121 - val_loss: 0.2801 - val_accuracy: 0.8833
Epoch 3/10
179/179 [==============================] - 11s 61ms/step - loss: 0.1076 - accuracy: 0.9655 - val_loss: 0.3805 - val_accuracy: 0.8565
Epoch 4/10
179/179 [==============================] - 9s 52ms/step - loss: 0.0601 - accuracy: 0.9818 - val_loss: 0.4429 - val_accuracy: 0.8438
Epoch 5/10
179/179 [==============================] - 11s 59ms/step - loss: 0.0353 - accuracy: 0.9909 - val_loss: 0.5175 - val_accuracy: 0.8502
Epoch 6/10
179/179 [==============================] - 10s 58ms/step - loss: 0.0270 - accuracy: 0.9930 - val_loss: 0.5644 - val_accuracy: 0.8517
Epoch 7/10
179/179 [==============================] - 9s 50ms/step - loss: 0.0184 - accuracy: 0.9956 - val_loss: 0.5577 - val_accuracy: 0.

In [161]:
y_pred_rnn = rnn_model.predict(x_test_padded)

50/50 [==============================] - 1s 6ms/step


In [162]:
y_pred_rnn[y_pred_rnn>0.5]=1
y_pred_rnn[y_pred_rnn<0.5]=0

In [166]:
accuracy_score(y_pred_rnn, y_test)

0.8472222222222222



---



<B><I>Here SVM with Bag of words has got higher accurancy of 89.27
.

## Prediction on New Data

In [193]:
def analyze_sentiment(new_tweet):
    sentiment_lbl = {1: 'Negative sentiment on the product', 0: 'Positive sentiment on the product'}

    new_tweets_bow = cv.transform([new_tweet])
    new_tweets_vec = [get_average_vector(new_tweet.split(), word2vec_model, 100)]
    new_tweets_vec = pd.DataFrame(new_tweets_vec).to_numpy()

    new_predictions_svm_bow = svm_bow_mdl.predict(new_tweets_bow)
    svm_bow_prediction_label = sentiment_lbl[new_predictions_svm_bow.item()]

    new_predictions_svm_w2v = svm_w2v_mdl.predict(new_tweets_vec)
    svm_w2v_prediction_label = sentiment_lbl[new_predictions_svm_w2v.item()]

    new_predictions_rf_bow = rf_bow_mdl.predict(new_tweets_bow)
    rf_bow_prediction_label = sentiment_lbl[new_predictions_rf_bow.item()]

    new_predictions_rf_w2v = rf_w2v_mdl.predict(new_tweets_vec)
    rf_w2v_prediction_label = sentiment_lbl[new_predictions_rf_w2v.item()]

    new_tweets_seq = tokenizer.texts_to_sequences([new_tweet])
    new_tweets_padded = pad_sequences(new_tweets_seq, maxlen=100)

    new_predictions_rnn = rnn_model.predict(new_tweets_padded, verbose=0)
    new_predictions_rnn[new_predictions_rnn > 0.5] = 1
    new_predictions_rnn[new_predictions_rnn < 0.5] = 0
    new_predictions_rnn = sentiment_lbl[new_predictions_rnn.item()]

    new_predictions_lstm = lstm_model.predict(new_tweets_padded, verbose=0)
    new_predictions_lstm[new_predictions_lstm > 0.5] = 1
    new_predictions_lstm[new_predictions_lstm < 0.5] = 0
    new_predictions_lstm = sentiment_lbl[new_predictions_lstm.item()]


    return {
        'SVM with Bag of Words Predictions': svm_bow_prediction_label,
        'SVM with Word2Vec Predictions': svm_w2v_prediction_label,
        'Random Forest with Bag of Words Predictions': rf_bow_prediction_label,
        'Random Forest with Word2Vec Predictions': rf_w2v_prediction_label,
        'RNN Predictions': new_predictions_rnn,
        'LSTM Predictions': new_predictions_lstm
    }


In [194]:
analyze_sentiment('This product is amazing!')

{'SVM with Bag of Words Predictions': 'Negative sentiment on the product',
 'SVM with Word2Vec Predictions': 'Positive sentiment on the product',
 'Random Forest with Bag of Words Predictions': 'Positive sentiment on the product',
 'Random Forest with Word2Vec Predictions': 'Positive sentiment on the product',
 'RNN Predictions': 'Negative sentiment on the product',
 'LSTM Predictions': 'Positive sentiment on the product'}

In [195]:
analyze_sentiment("I'm disappointed with the customer service.")

{'SVM with Bag of Words Predictions': 'Negative sentiment on the product',
 'SVM with Word2Vec Predictions': 'Positive sentiment on the product',
 'Random Forest with Bag of Words Predictions': 'Positive sentiment on the product',
 'Random Forest with Word2Vec Predictions': 'Positive sentiment on the product',
 'RNN Predictions': 'Negative sentiment on the product',
 'LSTM Predictions': 'Negative sentiment on the product'}